In [1]:
!nvidia-smi

Thu Sep 29 22:27:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth

## Install Requirements

In [2]:
!git clone https://github.com/ShivamShrirao/diffusers
!pip install git+https://github.com/ShivamShrirao/diffusers             # 2 times for now cause colab wasn't able to find it for some reason. Will fix later prolly.
%pip install -U --pre triton
%pip install accelerate
%pip install transformers
%pip install ftfy
%pip install bitsandbytes

Cloning into 'diffusers'...
remote: Enumerating objects: 7291, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 7291 (delta 103), reused 115 (delta 75), pack-reused 7136
Receiving objects: 100% (7291/7291), 2.23 MiB | 26.61 MiB/s, done.
Resolving deltas: 100% (4779/4779), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ShivamShrirao/diffusers to /tmp/pip-req-build-0rpb4ssu
  Running command git clone -q https://github.com/ShivamShrirao/diffusers /tmp/pip-req-build-0rpb4ssu
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 33.3 MB/s 
  Created wheel for diffusers: filename=diffusers-0.4.0.dev0-py3-none-any.whl size=229469 sha256=0e3b3f6d2cf407cfe46c6c279fafe753f644b25104570d05a038f022fffd4e13
  Stored in directory:

### Install xformers from precompiled wheel. (These were compiled on Tesla T4, thanks to https://github.com/metrolobo)

In [3]:
%pip install https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl

# if precompiled wheels don't work, install it with the following command. It will take around 40 minutes to compile.
# %pip install git+https://github.com/facebookresearch/xformers@1d31a3a#egg=xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102.9 MB 105 kB/s 


## Login to HF and run

In [4]:
%cd /content/diffusers/examples/dreambooth

/content/diffusers/examples/dreambooth


In [10]:
#@markdown Login to HuggingFace 🤗
from huggingface_hub import notebook_login
!git config --global credential.helper store
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [7]:
#@markdown Name/Path of the initial model.
MODEL_NAME = "CompVis/stable-diffusion-v1-4" #@param {type:"string"}

#@markdown Path for images of the concept for training.
INSTANCE_DIR = "/content/data/sks" #@param {type:"string"}
!mkdir -p $INSTANCE_DIR

#@markdown A general name for class like dog for dog images.
CLASS_NAME = "guy" #@param {type:"string"}
CLASS_DIR = f"/content/data/{CLASS_NAME}"

#@markdown Where to save the model after training.
OUTPUT_DIR = "/content/models/sks" #@param {type:"string"}

#@markdown sks is a rare identifier, feel free to replace it.

In [ ]:
#@markdown Upload your images (5-6) by running this cell.

#@markdown OR

#@markdown You can use the file manager on the left panel to upload to INSTANCE_DIR (it uploads a bit faster)

import os
from google.colab import files
import shutil

uploaded = files.upload()
for filename in uploaded.keys():
    dst_path = os.path.join(INSTANCE_DIR, filename)
    shutil.move(filename, dst_path)

### Start training

remove `--use_8bit_adam` flag in the cell below if you got more than 18 GB VRAM.

`--mixed_precision="fp16"` uses a little more VRAM for some reason but is 1.5-2x faster on GPUs with tensor cores, change it to `--mixed_precision="no"` if you face out of memory issues.

In [11]:
!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME --use_auth_token \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="photo of sks {CLASS_NAME}" \
  --class_prompt="photo of a {CLASS_NAME}" \
  --seed=1337 \
  --resolution=512 \
  --center_crop \
  --train_batch_size=1 \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=12 \
  --sample_batch_size=4 \
  --max_train_steps=800



Downloading:   3% 87.5M/3.44G [00:01<00:51, 64.8MB/s]
Downloading:   3% 94.3M/3.44G [00:01<00:55, 59.9MB/s]
Downloading:   3% 102M/3.44G [00:01<00:50, 65.4MB/s] 
Downloading:   3% 111M/3.44G [00:01<00:47, 69.4MB/s]
Downloading:   3% 118M/3.44G [00:01<00:46, 70.6MB/s]
Downloading:   4% 125M/3.44G [00:01<00:48, 68.7MB/s]
Downloading:   4% 133M/3.44G [00:01<00:46, 71.2MB/s]
Downloading:   4% 140M/3.44G [00:02<00:45, 71.9MB/s]
Downloading:   4% 148M/3.44G [00:02<00:44, 74.2MB/s]
Downloading:   5% 156M/3.44G [00:02<00:45, 72.2MB/s]
Downloading:   5% 164M/3.44G [00:02<00:43, 74.9MB/s]
Downloading:   5% 172M/3.44G [00:02<00:43, 75.9MB/s]
Downloading:   5% 180M/3.44G [00:02<00:41, 77.7MB/s]
Downloading:   5% 188M/3.44G [00:02<00:43, 74.6MB/s]
Downloading:   6% 196M/3.44G [00:02<00:42, 75.7MB/s]
Downloading:   6% 204M/3.44G [00:02<00:41, 77.5MB/s]
Downloading:   6% 212M/3.44G [00:02<00:40, 79.1MB/s]
Downloading:   6% 220M/3.44G [00:03<00:43, 73.3MB/s]
Downloading:   7% 228M/3.44G [00:03<00:42,

## Save trained model to google drive (Optional)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

#@markdown Enter the path in your google drive to save model at.

Destination = "stable_diffusion_weights/" #@param {type:"string"}

Destination = "/content/drive/MyDrive/" + Destination

!mkdir -p $Destination
!cp -ra $OUTPUT_DIR $Destination

## Inference

In [18]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from IPython.display import display

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [19]:
pipe = StableDiffusionPipeline.from_pretrained(OUTPUT_DIR, torch_dtype=torch.float16).to("cuda")

In [20]:
g_cuda = torch.Generator(device='cuda')
# g_cuda.manual_seed(1337)

In [ ]:
prompt = "photo of sks guy, digital painting" #@param {type:"string"}
num_samples = 4 #@param {type:"number"}

with autocast("cuda"), torch.no_grad():
    images = pipe([prompt] * num_samples, height=512, width=512, num_inference_steps=50, guidance_scale=7.5, generator=g_cuda).images

for img in images:
    display(img)

  0%|          | 0/51 [00:00<?, ?it/s]